In [ ]:
# count number users
print(df['ID'].count())

# count defaults
print(df[df['default.payment.next.month']==1]['ID'].count())

# if the user has no bill payment at all
print(df[(df.BILL_AMT1==0) \
        &(df.BILL_AMT2==0) \
        &(df.BILL_AMT3==0) \
        &(df.BILL_AMT4==0) \
        &(df.BILL_AMT5==0) \
        &(df.BILL_AMT6==0) \
        & (df['default.payment.next.month']==1)
      ]['ID'].count())

# if the user pay more than their were ask to pay
print(df[(df.PAY_AMT1>=df.BILL_AMT1) \
        &(df.PAY_AMT2>=df.BILL_AMT2) \
        &(df.PAY_AMT3>=df.BILL_AMT3) \
        &(df.PAY_AMT4>=df.BILL_AMT4) \
        &(df.PAY_AMT5>=df.BILL_AMT5) \
        &(df.PAY_AMT6>=df.BILL_AMT6) \
        & (df['default.payment.next.month']==1)
      ]['ID'].count())

# if the user has bigger bill than their credit limit
print(df[((df.LIMIT_BAL<=df.BILL_AMT1) \
        |(df.LIMIT_BAL<=df.BILL_AMT2) \
        |(df.LIMIT_BAL<=df.BILL_AMT3) \
        |(df.LIMIT_BAL<=df.BILL_AMT4) \
        |(df.LIMIT_BAL<=df.BILL_AMT5) \
        |(df.LIMIT_BAL<=df.BILL_AMT6)) \
        & (df['default.payment.next.month']==1)
      ]['ID'].count())

Note that 
- the data dictionary the the README file is missing the item "PAY_1". I have, therefore, added this missing item into the data dictionary.
- "PAY_0" was defined as repayment status in September. I believe it was a mislabelled. "PAY_1" should be the status in September (consistent with "BILL_AMT#" and "PAY_AMT#). Then, "PAY_0" should be the status in October. 




In [ ]:

# To understand them, 
col_names = []
for i in range(1,6):
    col_names.append('BILL_AMT'+str(i))
    col_names.append('PAY_AMT'+str(i))
    col_names.append('PAY_'+str(i))

print(df_explore[np.array(col_names)].sample(10))




sns.distplot( df_explore.loc[df_explore.DEFAULT_PAY==0,'AGE'], color='skyblue', label='ok')
sns.distplot( df_explore.loc[df_explore.DEFAULT_PAY==1,'AGE'], color='red', label='default')